In [1]:
%run ../spark-default.py

In [2]:
import builtins as b
from datetime import datetime, timedelta, date
import sys, traceback
from delta.tables import DeltaTable

from scipy.cluster.hierarchy import linkage, dendrogram
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline

In [3]:
df = spark.table("stage.ciha_step3")

df.printSchema()

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_atend_dow: integer (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- 

In [4]:
df

mes_cmpt,espec,cgc_hosp,munic_res,nasc,sexo,uti_mes_to,uti_int_to,proc_rea,qt_proc,dt_atend,dt_atend_dow,dt_saida,diag_princ,diag_secun,cobranca,natureza,gestao,munic_mov,cod_idade,idade,dias_perm,morte,nacional,car_int,homonimo,cnes,fonte,cgc_consor,modalidade,input_file_name,arquivo_sigla_uf,ano_cmpt,munic_res_nome_uf,munic_mov_nome_uf,munic_res_nome_municipio,munic_mov_nome_municipio,munic_res_sigla_uf,munic_mov_sigla_uf,munic_res_regiao,munic_mov_regiao,munic_res_populacao_residente,munic_mov_populacao_residente,munic_res_area_unidade_territorial,munic_mov_area_unidade_territorial,feriado,feriado_distancia,feriado_info,diag_princ_sub_categoria,diag_princ_sub_categoria_classificacao,diag_princ_restringido_sexo,diag_princ_causa_obito,diag_princ_sub_categoria_descricao,diag_princ_sub_categoria_descricao_abreviada,diag_princ_sub_categoria_referencia,diag_princ_sub_categoria_excluidos,diag_princ_categoria,diag_princ_categoria_classificacao,diag_princ_categoria_descricao,diag_princ_categoria_descricao_abreviada,diag_princ_categoria_referencia,diag_princ_capitulo_numero,diag_princ_capitulo_descricao,diag_princ_capitulo_descricao_abreviada,diag_secun_sub_categoria,diag_secun_sub_categoria_classificacao,diag_secun_restringido_sexo,diag_secun_causa_obito,diag_secun_sub_categoria_descricao,diag_secun_sub_categoria_descricao_abreviada,diag_secun_sub_categoria_referencia,diag_secun_sub_categoria_excluidos,diag_secun_categoria,diag_secun_categoria_classificacao,diag_secun_categoria_descricao,diag_secun_categoria_descricao_abreviada,diag_secun_categoria_referencia,diag_secun_capitulo_numero,diag_secun_capitulo_descricao,diag_secun_capitulo_descricao_abreviada,munic_res_idhm,munic_mov_idhm,proc_nome,proc_tp_complexidade,proc_tp_sexo,proc_qt_max_exec,proc_qt_dias_perm,proc_qt_pontos,proc_ds_procedimento,proc_no_grupo,proc_no_sub_grupo,proc_no_forma_organizacao
01,null,76613835000189,410690,1996-05-19,1,0,000,0301060002,1,2019-01-09,4,2019-01-09,Z000,null,null,null,D,410690,4,22,0,0,null,null,null,0015334,01,1,01,s3a://datalake/so...,PR,2019,Paraná,Paraná,Curitiba,Curitiba,PR,PR,Sul,Sul,1751907,1751907,435.036,435.036,false,5,"[{2019-01-14, Vés...",Z000,null,null,null,Exame médico geral,Z00.0 Exame medic...,null,null,Z00,null,Exame geral e inv...,Z00 Exame geral...,null,21,Capítulo XXI - Fa...,XXI. Contatos com...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.823,0.823,null,null,null,null,null,null,null,null,null,null
08,null,59981712000181,354980,1951-02-01,3,0,000,0205010040,1,2019-08-07,4,2019-08-07,I10,null,null,null,M,354980,4,68,0,0,null,null,null,2798298,06,6,01,s3a://datalake/so...,SP,2019,São Paulo,São Paulo,São José do Rio P...,São José do Rio P...,SP,SP,Sudeste,Sudeste,408258,408258,431.963,431.963,false,12,"[{2019-07-26, Fun...",I10,null,null,null,Hipertensão essen...,I10 Hipertensao...,null,null,I10,null,Hipertensão essen...,I10 Hipertensao...,null,9,Capítulo IX - Doe...,IX. Doenças do a...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.797,0.797,ULTRASSONOGRAFIA ...,2,I,0005,9999,0000,É O MÉTODO MAIS I...,Procedimentos com...,Diagnóstico em la...,Exames hematológi...
02,null,32190092000378,330580,2002-05-25,3,0,000,0303010061,1,2019-02-06,4,2019-02-10,R509,null,11,null,M,330580,4,16,4,0,null,null,null,2297795,01,1,02,s3a://datalake/so...,RJ,2019,Rio de Janeiro,Rio de Janeiro,Teresópolis,Teresópolis,RJ,RJ,Sudeste,Sudeste,163746,163746,770.601,770.601,false,22,"[{2019-02-28, Fac...",R509,null,null,null,Febre não especif...,R50.9 Febre NE,null,null,R50,null,Febre de origem d...,R50 Febre orige...,null,18,Capítulo XVIII - ...,XVIII.Sint sinais...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.73,0.73,TRATAMENTO DE DOE...,2,I,0001,0004,0080,TRATAMENTO CLÍNIC...,Procedimentos clí...,Tratamentos clíni...,Tratamento de doe...
11,null,87873279000368,432280,1940-12-26,3,0,000,0415040000,3,2019-11-29,6,2019-11-29,S832,null,null,null,

In [5]:
df_vol = df.groupBy(
    col("dt_atend")
).agg(
    count("*").alias("total_atendimentos"),
    sum(when(col("sexo") == lit("1"), lit(1)).otherwise(lit(0))).alias("total_atendimentos_masc"),
    sum(when(col("sexo") == lit("3"), lit(1)).otherwise(lit(0))).alias("total_atendimentos_fem")
)

In [6]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.ciha_atendimentos") \
  .addColumns(df_vol.schema) \
  .execute()

In [7]:
df_vol.write.mode("overwrite").insertInto("stage.ciha_atendimentos")

In [8]:
spark.stop()